In [4]:
import pandas as pd
import ast
from funciones import extract
import pyarrow
import json
import gzip
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
%load_ext memory_profiler
import pyarrow.parquet as pq


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [5]:
df = pd.read_csv(r'data_set_limpio/games_preparado.csv.gz')

peak memory: 152.62 MiB, increment: 22.25 MiB


In [7]:
item = pd.read_csv(r'data_set_limpio/items_preparado.csv.gz')

peak memory: 1628.27 MiB, increment: 1484.53 MiB


In [9]:
opinion = pd.read_csv(r'data_set_limpio/reviews_preparado.csv.gz')

peak memory: 1188.91 MiB, increment: 49.68 MiB


In [9]:
def developer(x):
    if x not in list(df['developer']):
        return 'El desarrollador no se encuentra en la base de datos'
    
    fecha_inicio = df.loc[df['developer'] == x]['release_date'].min()[:4]
    fecha_final =  df.loc[df['developer'] == x]['release_date'].max()
    
    anio = {}
    
    free = {}
    
    while fecha_inicio <= fecha_final:
        if len(df[(df['release_date'] >= fecha_inicio) & (df['release_date'] <= fecha_inicio[:4]+'-12-31') & (df['developer'] == x)]) > 0:
            anio[fecha_inicio[:4]] = len(df[(df['release_date'] >= fecha_inicio) & (df['release_date'] <= fecha_inicio[:4]+'-12-31') & (df['developer'] == x)])
            free[fecha_inicio[:4]] = len(df[(df['release_date'] >= fecha_inicio) & (df['release_date'] <= fecha_inicio[:4]+'-12-31') & (df['developer'] == x) & ((df['price'] == 'Free') | (df['price'] == 'Free to Play') )])
        fecha_inicio = str(int(fecha_inicio[:4])+1)+'-01-01'


    for x,y  in free.items():
        free[x] = str(round((y / anio[x])*100,2))+'%'


    resultado = {'Cantidad de Items': anio,'Contenido Free':free}

    
    return resultado




In [29]:
developer('Valve')

{'Cantidad de Items': {'1998': 1,
  '1999': 1,
  '2000': 2,
  '2001': 1,
  '2003': 1,
  '2004': 5,
  '2006': 2,
  '2007': 3,
  '2008': 1,
  '2009': 1,
  '2010': 2,
  '2011': 1,
  '2012': 1,
  '2017': 2},
 'Contenido Free': {'1998': '0.0%',
  '1999': '0.0%',
  '2000': '0.0%',
  '2001': '0.0%',
  '2003': '0.0%',
  '2004': '0.0%',
  '2006': '0.0%',
  '2007': '33.33%',
  '2008': '0.0%',
  '2009': '0.0%',
  '2010': '50.0%',
  '2011': '0.0%',
  '2012': '0.0%',
  '2017': '0.0%'}}

In [7]:
def userdata(user: str):
    
    try:
        for x in pd.read_csv(r'data_set_limpio/items_preparado.csv.gz', chunksize=3000):
            if user in list(x['user_id']):
                aux = x
                break
            x = 0
        
        
        
        precios = []
        respuesta = {}
        
        
        usuario = aux.loc[aux['user_id'] == user]['items']
        
        
        if not usuario.empty:
            usuario = usuario.iloc[0]
        data = ast.literal_eval(usuario)
        result = pd.DataFrame(data)
        result.dropna(inplace=True)
        

        for y in result['item_name']:
            price = df.loc[df['app_name'] == y]['price'].values
            if len(price) > 0:
                try: 
                    price_value = float(price[0])
                    precios.append(price_value)
                except ValueError:
                    pass  
                
        respuesta['Usuario'] = user
        respuesta['Dinero gastado'] = str(round(sum(precios)))+' USD'
        respuesta["cantidad de items"] = str(len(result))
        
        respuesta['Porcentaje de recomendaciones'] = str(round(((len(opinion.loc[opinion['user'] == user]) / len(result))) * 100,2))+ '%'
        
        return respuesta
    except:
        return 'El usuario no se encuentra en la base de datos.'

In [30]:

userdata('76561197970982479')


{'Usuario': '76561197970982479',
 'Dinero gastado': '2315 USD',
 'cantidad de items': '277',
 'Porcentaje de recomendaciones': '1.08%'}

In [184]:
def best_developer_year(year):
    
    anio = pd.read_csv('data_set_limpio\Max_developer_year.csv')
    
    anio = anio.sort_values('Anio', ascending=False)
    
    seleccion = anio.loc[anio['Anio'] == year].copy()
    
    seleccion.fillna('No existen resenias',inplace=True)
    
    respuesta = {}
    
    respuesta['Anio'] = seleccion['Anio'].values[0]
    respuesta['Top 1'] = seleccion['Top 1'].values[0]
    respuesta['Top 2'] = seleccion['Top 2'].values[0]
    respuesta['Top 3'] = seleccion['Top 3'].values[0]
    return respuesta

In [1]:
def resenias_developer( des : str ):
    
    try:

        table = pq.read_table('data_set_limpio/recomends_dev.parquet')
        
        dev = table.to_pandas()
        
        developer = dev.loc[dev['Developers'] == des]
        
        respuesta = {'Deloper' : str(developer['Developers'].values[0]), 'Reviews positivos' : str(developer['Positivo'].values[0]),'Reviews negativos' : str(developer['Negativo'].values[0])}
        dev = 0
        return respuesta
    
    except:
        
        return 'No ingreso un valor relevante, o el desarrollador no se encuentra en la base de datos'   

In [10]:
resenias_developer('Valve')

{'Deloper': 'Valve', 'Reviews positivos': '5857', 'Reviews negativos': '2741'}